In [ ]:
import os
import tweepy
import logging
import pandas as pd
import random
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
class Twitter:
    # Variables that contains the credentials to access Twitter API
    ACCESS_TOKEN = '2214038774-bRdjyYxzju7EzDC936E9Lw8GJPH1vvc5QG3urDV'
    ACCESS_SECRET = 'kADbwwpPNr19yzSwSWh0xua9ghtEQtFtf4NJdx1zZcnm5'
    CONSUMER_KEY = 'cDOqVRVhKcg2uFzcXHGrdttXC'
    CONSUMER_SECRET = 'Mab1X0vQg3isdXcjfxebPod6NV1VwNUDPOAevzA6yM0TVDjC2F'

    # Setup access to API
    def connect_to_twitter_OAuth(self):
        auth = tweepy.OAuthHandler(Twitter.CONSUMER_KEY, Twitter.CONSUMER_SECRET)
        auth.set_access_token(Twitter.ACCESS_TOKEN, Twitter.ACCESS_SECRET)
        self.api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
        logger = logging.getLogger()
        try:
            self.api.verify_credentials()
        except Exception as e:
            logger.error("Error creating API", exc_info=True)
            raise e
        logger.info("API created")
        return self.api


    def get_recent_tweet(self, query, language, count=100):
        query = query + " -filter:retweets"
        fetched_tweets = tweepy.Cursor(self.api.search, q=query, lang=language, tweet_mode='extended', result_type='recent').items(count)
        users_tweets = list()
        for tweet in fetched_tweets:
            users_tweets.append([tweet.user.screen_name, tweet.full_text, tweet.created_at, tweet.lang,tweet.favorite_count,tweet.retweet_count])
        df = pd.DataFrame(data=users_tweets, columns=["user", "text", "created_at", "lang","favs","retweets"])
        return df
    

    def get_user_tweet(self, username, count=100):
        fetched_tweets = tweepy.Cursor(self.api.user_timeline, screen_name=username, exclude_replies=True, tweet_mode='extended').items(count)
        users_tweets = list()
        for tweet in fetched_tweets:
            users_tweets.append([tweet.user.screen_name, tweet.full_text, tweet.created_at, tweet.lang])
        df = pd.DataFrame(data=users_tweets, columns=["user", "text", "created_at", "lang"])
        return df


    def df_to_csv(self, df):
        df.to_csv("tweets.csv", encoding="utf-8")


In [ ]:
twitter = Twitter()
twitter.connect_to_twitter_OAuth()
data = twitter.get_recent_tweet(query="اسنپ", language="fa", count=2000)
# user_data = twitter.get_user_tweet(username='mh___mp98', count=50)
# twitter.df_to_csv(data)

In [ ]:
sentiment_options = ['positive','negative','neutral']
data['sentiment'] = data['user'].apply(lambda x:random.choice(sentiment_options))

In [ ]:
data['created_at'].min(),data['created_at'].max()

(Timestamp('2021-01-23 13:21:44'), Timestamp('2021-01-28 09:57:32'))

In [ ]:
def tweets_dist(data):
    fig = go.Figure()

    fig.add_trace(go.Histogram(
      x=data['created_at'],
      nbinsx=30
    ))

    fig.update_layout(
      title_text='Distribution of tweets over time',
      xaxis_title_text='Time',
      yaxis_title_text='Frequency',
      bargap=0.1,
      bargroupgap=0.2)
    
    return fig

#sample use
tweets_dist(data).show()

In [ ]:
grouping_freq='20min'

In [ ]:
grouped_negative = data[data['sentiment'] == 'negative'].groupby(pd.Grouper(key='created_at',freq=grouping_freq)).count()
grouped_neutral = data[data['sentiment'] == 'neutral'].groupby(pd.Grouper(key='created_at',freq=grouping_freq)).count()
grouped_positive = data[data['sentiment'] == 'positive'].groupby(pd.Grouper(key='created_at',freq=grouping_freq)).count()
grouped_total = data.groupby(pd.Grouper(key='created_at',freq=grouping_freq)).count()
final_data = pd.DataFrame(data={
    'negative': grouped_negative['sentiment'],
    'neutral' : grouped_neutral['sentiment'],
    'positive': grouped_positive['sentiment'],
},index=grouped_total.index)


In [ ]:
final_data.reset_index(inplace=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=final_data['created_at'],
    y=final_data['neutral'],
    name="Neutrals",
    opacity=0.8,
    mode='lines',
    line=dict(width=0.5, color='rgb(131, 90, 241)'),
    stackgroup='one' 
))
fig.add_trace(go.Scatter(
    x=final_data['created_at'],
    y=final_data['negative'],
    name="Negatives",
    opacity=0.8,
    mode='lines',
    line=dict(width=0.5, color='rgb(255, 50, 50)'),
    stackgroup='two' 
))
fig.add_trace(go.Scatter(
    x=final_data['created_at'],
    y=final_data['positive'],
    name="Positives",
    opacity=0.8,
    mode='lines',
    line=dict(width=0.5, color='rgb(184, 247, 212)'),
    stackgroup='three' 
))
fig.update_layout(
    title_text='Distribution of sentiments over time',
    xaxis_title_text='Time',
    yaxis_title_text='Frequency'
)
fig.update_xaxes(
    rangeslider_visible=True
)
fig.show()


In [ ]:
def sentiment_dist(data,grouping_freq='10min'):
  grouped_negative = data[data['sentiment'] == 'negative'].groupby(pd.Grouper(key='created_at',freq=grouping_freq)).count()
  grouped_neutral = data[data['sentiment'] == 'neutral'].groupby(pd.Grouper(key='created_at',freq=grouping_freq)).count()
  grouped_positive = data[data['sentiment'] == 'positive'].groupby(pd.Grouper(key='created_at',freq=grouping_freq)).count()
  grouped_total = data.groupby(pd.Grouper(key='created_at',freq=grouping_freq)).count()
  final_data = pd.DataFrame(data={
      'negative': grouped_negative['sentiment'],
      'neutral' : grouped_neutral['sentiment'],
      'positive': grouped_positive['sentiment'],
  },index=grouped_total.index)
  final_data.reset_index(inplace=True)
  fig = go.Figure()
  fig.add_trace(go.Scatter(
      x=final_data['created_at'],
      y=final_data['neutral'],
      name="Neutrals",
      opacity=0.8,
      mode='lines',
      line=dict(width=0.5, color='rgb(131, 90, 241)'),
      stackgroup='one' 
  ))
  fig.add_trace(go.Scatter(
      x=final_data['created_at'],
      y=final_data['negative'],
      name="Negatives",
      opacity=0.8,
      mode='lines',
      line=dict(width=0.5, color='rgb(255, 50, 50)'),
      stackgroup='two' 
  ))
  fig.add_trace(go.Scatter(
      x=final_data['created_at'],
      y=final_data['positive'],
      name="Positives",
      opacity=0.8,
      mode='lines',
      line=dict(width=0.5, color='rgb(184, 247, 212)'),
      stackgroup='three' 
  ))
  fig.update_layout(
      title_text='Distribution of sentiments over time',
      xaxis_title_text='Time',
      yaxis_title_text='Frequency'
  )
  fig.update_xaxes(
      rangeslider_visible=True
  )
  return fig